## Male CNS - BANC landmarks
In this notebook, we will generate a "shortcut" to transform data between BANC and the male CNS. The general idea is:
1. Evenly sample points in male CNS
2. Transform the VNC and the brain part via their respective JRC2018 templates to the BANC template

In [1]:
import numpy as np
import pandas as pd

import cloudvolume as cv

import flybrains
import navis

In [9]:
del flybrains.JRCFIB2022M._mesh_vnc

In [10]:
# Grab the meshes for brain and VNC
brain = flybrains.JRCFIB2022M.mesh_brain
vnc = flybrains.JRCFIB2022M.mesh_vnc

In [11]:
all_verts = np.vstack([brain.vertices, vnc.vertices])
bounds = np.vstack([all_verts.min(axis=0), all_verts.max(axis=0)])
bounds

array([[  45319.40625  ,   38146.7109375,   79817.3828125],
       [ 726917.875    ,  550810.25     , 1072833.875    ]])

In [12]:
# Add 10 microns around the mesh
offset = 10000
bounds_off = bounds.astype(int)
bounds_off[0, :] -= offset
bounds_off[1, :] += offset
bounds_off

array([[  35319,   28146,   69817],
       [ 736917,  560810, 1082833]])

In [13]:
# Sample points within the volume
res = 30000
sample = (
    np.mgrid[
        bounds_off[0, 0] : bounds_off[1, 0] : res,
        bounds_off[0, 1] : bounds_off[1, 1] : res,
        bounds_off[0, 2] : bounds_off[1, 2] : res,
    ]
    .reshape(3, -1)
    .T
)

# Drop points outside the volume
sample = sample[navis.in_volume(sample, brain) | navis.in_volume(sample, vnc)]
sample.shape

(1170, 3)

In [15]:
# Next, we have to split the sampled points into:
# 1. the brain part which we will mirror via JRC2018M
# 2. the VNC part which we will mirror via JRCVNC2018M
is_brain = navis.in_volume(sample, brain)
is_vnc = navis.in_volume(sample, vnc)

# Make sure no point is in both
(is_brain & is_vnc).sum()

np.int64(0)

In [30]:
# Transform the points
sample_xf_brain = navis.xform_brain(sample[is_brain], source="JRCFIB2022M", target="JRC2018M")
sample_xf_brain = navis.xform_brain(sample_xf_brain, source="JRC2018M", target="JRC2018F")
sample_xf_brain = navis.xform_brain(sample_xf_brain, source="JRC2018F", target="BANC")

Transform path: JRCFIB2022M -> JRCFIB2022Mum -> JRC2018M
Transform path: JRC2018M -> JRC2018U -> JRC2018F
Transform path: JRC2018F -> BANCum -> BANC


In [ ]:
# Note: we have to avoid navis trying to go via FlyWire -> JRC2018F
# We have to make it a multi-step process
sample_xf_vnc = navis.xform_brain(sample[is_vnc], source="JRCFIB2022M", target="MANC")
sample_xf_vnc = navis.xform_brain(sample_xf_vnc, source="MANC", target="JRCVNC2018M")
sample_xf_vnc = navis.xform_brain(sample_xf_vnc, source="JRCVNC2018M", target="JRCVNC2018F")
sample_xf_vnc = navis.xform_brain(sample_xf_vnc, source="JRCVNC2018F", target="BANC")

Transform path: JRCFIB2022M -> JRCFIB2022Mum -> MANCum-JRCFIB2022Mum(post) -> MANCum -> MANC
Transform path: MANC -> MANCum -> JRCVNC2018M
Transform path: JRCVNC2018M -> JRCVNC2018U -> JRCVNC2018F
Transform path: JRCVNC2018F -> BANCum -> BANC


In [ ]:
# This also looks decent
navis.plot3d([flybrains.BANC.mesh_whole_brain, sample_xf_brain, sample_xf_vnc])

In [32]:
# Bring it together
source = pd.DataFrame(np.vstack((sample[is_brain], sample[is_vnc])), columns=['x_mcns', 'y_mcns', 'z_mcns']).round().astype(int)
target = pd.DataFrame(np.vstack((sample_xf_brain, sample_xf_vnc)), columns=['x_banc', 'y_banc', 'z_banc']).round().astype(int)
lm = pd.concat((source, target), axis=1)
lm.head()

,x_mcns,y_mcns,z_mcns,x_banc,y_banc,z_banc
0,65319,208146,279817,118834,142836,145730
1,65319,208146,309817,122311,171460,152858
2,65319,238146,279817,118903,156510,124176
3,65319,238146,309817,122146,183473,132195
4,65319,268146,279817,115745,167695,104015


In [33]:
lm.to_csv('maleCNS_BANC_landmarks_nm.csv', index=False)